In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import duckdb
import streamlit as st
from datetime import datetime, timedelta


In [6]:
def introduce_typo(value):
    """Thêm lỗi chính tả ngẫu nhiên vào một chuỗi với ký tự đặc biệt hoặc số."""
    if isinstance(value, str) and len(value) > 3 and np.random.rand() < 0.3:  # 30% lỗi chính tả
        pos = np.random.randint(0, len(value))
        typo_char = np.random.choice(list('!@#&$^(*0123456789'))  # Chọn ký tự đặc biệt hoặc số
        return value[:pos] + typo_char + value[pos+1:]
    return value

In [7]:
def randomize_date(index):
    """Tạo ngày ngẫu nhiên trong khoảng 2 năm từ ngày hiện tại để tăng độ đa dạng."""
    base_date = datetime(2022, 1, 1)
    random_days = random.randint(-365, 365)  # Ngày lệch ngẫu nhiên rộng hơn
    new_date = base_date + timedelta(days=index + random_days)
    return new_date.strftime("%Y-%m-%d")

In [8]:
def raw_data(input_csv, output_csv):
    # Đọc dữ liệu
    df = pd.read_csv(input_csv)
    
    # Kiểm tra cột hợp lệ trước khi raw hóa
    print("Available columns before modification:", df.columns)
    
    # Danh sách cột có thể bị ẩn một phần
    potential_hide_columns = [col for col in ["Date", "event", "fit Man", "Client", "City"] if col in df.columns]
    
    # Chọn ngẫu nhiên 40% số vị trí trong cột để thay bằng "####"
    if potential_hide_columns:
        for col in potential_hide_columns:
            mask = np.random.rand(len(df)) < 0.4  # 40% giá trị trong cột bị thay thế
            df.loc[mask, col] = "####"
    
    # Ngẫu nhiên hóa ngày trong cột "Date"
    if "Date" in df.columns:
        df["Date"] = [randomize_date(i) for i in range(len(df))]
    
    # Fake Status và cập nhật Courier Status
    status_choices = ["Pending", "Processing", "Awaiting Shipment", "Shipped", "Out for Delivery", "Delivered", "Cancelled", "Returned to Seller", "Rejected by Buyer", "Lost in Transit", "Damaged in Transit"]
    probabilities = [0.12, 0.1, 0.08, 0.28, 0.1, 0.14, 0.06, 0.05, 0.03, 0.02, 0.02]  # Xác suất xảy ra
    df["Status"] = np.random.choice(status_choices, size=len(df), p=probabilities)
    
    # Mapping giữa Status và Courier Status
    status_to_courier = {
        "Pending": "Not Shipped",
        "Processing": "Not Shipped",
        "Awaiting Shipment": "Not Shipped",
        "Shipped": "In Transit",
        "Out for Delivery": "Out for Delivery",
        "Delivered": "Delivered",
        "Cancelled": "N/A",
        "Returned to Seller": "Returned",
        "Rejected by Buyer": "Returned",
        "Lost in Transit": "Lost",
        "Damaged in Transit": "Damaged"
    }
    df["Courier Status"] = df["Status"].map(status_to_courier)
    
    # Thêm lỗi chính tả vào cột "Contact", "Department" với xác suất cao hơn
    for col in ["Contact", "Department"]:
        if col in df.columns:
            df[col] = df[col].apply(introduce_typo)
    
    # Xóa ngẫu nhiên 20% dữ liệu trong cột Payment, Profit để tăng độ khó
    for col in ["Payment", "Profit"]:
        if col in df.columns:
            mask = np.random.rand(len(df)) < 0.2  # 20% dữ liệu bị thiếu
            df.loc[mask, col] = np.nan
    
    # Biến đổi số lượng sản phẩm (Qty) một cách mạnh hơn
    if "Qty" in df.columns:
        df["Qty"] = df["Qty"].apply(lambda x: x + np.random.randint(0, 6) if np.random.rand() < 0.5 else x)
    
    # Kiểm tra lại số cột trước khi lưu
    print("Columns before saving:", df.columns)
    
    # Lưu lại file CSV đã raw hóa
    df.to_csv(output_csv, index=False)
    print(f"Dữ liệu đã raw hóa được lưu tại: {output_csv}")


In [9]:
raw_data(r"C:\Users\Admin\Desktop\E_commerce_proj\data\raw\Amazon_Sale_Report_Raw_origin.csv", r"C:\Users\Admin\Desktop\E_commerce_proj\data\raw\testdata.csv")

Available columns before modification: Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by'],
      dtype='object')
Columns before saving: Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by'],
      dtype='object')
Dữ liệu đã raw hóa được lưu tại: C:\Users\Admin\Desktop\E_commerce_proj\data\raw\testdata.csv


FAKE MỚI